In [38]:
pip install cdo

Looking in indexes: https://metoffice.jfrog.io/metoffice/api/pypi/pypi/simple
You should consider upgrading via the '/home/h03/hgilmour/.conda/envs/myclone/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import iris
import numpy as np
import pandas as pd
import cdo
from cdo import *
import xarray as xr

In [2]:
#cdo = Cdo()

Things to re-grid:
1) CPM segmentation.nc files 1998-2007 (to 10km IMERG using nearest neighbour)
2) Satellite segmentation.nc files 2001-2007 (to 10km IMERG using nearest neighbour)
3) CPM Tb 1998-2007 (to 10km IMERG using remapcon)
4) Satellite Tb 2001-2007 (to 10km IMERG using remapcon)
5) CPM precip 1998-2007 (to 10m IMERG using remapcon)
6) CPM w 1998-2007 (to 10m IMERG using remapcon)

**1) Re-grid CPM segmentation.nc files 1998-2007 (to 10km IMERG using nearest neighbour**)

In [38]:
## Load in CPM segmentation file for each year ###
mask_file_1999 = '/data/users/hgilmour/initial_tracks/tobac_initial_tracks/segmentation/segmentation_yearly_1999.nc'

In [39]:
### Extract the first timestep of the segmentation.nc file for testing ##

mask_1999 = iris.load_cube(mask_file_1999)
mask_1999
#subset = mask_1999[0:1,:,:]

/var/tmp/ipykernel_20211/408015420.py:3: FutureWarning: Ignoring a datum in netCDF load for consistency with existing behaviour. In a future version of Iris, this datum will be applied. To apply the datum when loading, use the iris.FUTURE.datum_support flag.
  mask_1999 = iris.load_cube(mask_file_1999)


<iris 'Cube' of segmentation_mask / (1) (time: 8760; -- : 1360; -- : 1360)>

In [40]:
## Also load in the Tb CPM file to get the lat lon info from ##
tb_file = '/data/users/hgilmour/tb/tb_1999.nc'

tb_1999 = iris.load(tb_file)

tb_1999 = tb_1999[1]

/var/tmp/ipykernel_20211/1341759942.py:4: FutureWarning: Ignoring a datum in netCDF load for consistency with existing behaviour. In a future version of Iris, this datum will be applied. To apply the datum when loading, use the iris.FUTURE.datum_support flag.
  tb_1999 = iris.load(tb_file)


In [41]:
#tb_1999_subset = tb_1999[0:1,:,:]

In [ ]:
## Extract the data from the segmentation mask ##

mask_1999_data = mask_1999.data

In [ ]:
## creating a new black cube to put data into ##
TM = mask_1999.coord('time')
hrtime = TM.points

nt,nx,ny = mask_1999_data.shape
subset_new = np.zeros((nt,nx,ny))

TimeCoords, LatCoords, LonCoords = mask_1999.coord('time'), tb_1999.coord('projection_y_coordinate'), tb_1999.coord('projection_x_coordinate') 
vrbcoords=[(TimeCoords, 0), (LatCoords, 1), (LonCoords, 2)]

## Instantiate new cube 
newcube = iris.cube.Cube(subset_new,standard_name=mask_1999.standard_name, long_name=mask_1999.long_name, var_name=mask_1999.var_name, units=mask_1999.units, attributes=mask_1999.attributes, dim_coords_and_dims=vrbcoords)


In [ ]:
## Put data from mask file into the new empty cube ##
newcube.data = mask_1999.data

In [ ]:
## testing to see if new masked data has been added to the cube ##

newcube[0,1:100,1:100].data

In [ ]:
# savefile = '/data/users/hgilmour/initial_tracks/tobac_initial_tracks/segmentation/coords_test_segmentation_yearly_1999.nc'

# iris.save(newcube, savefile)

In [ ]:
newcube

In [ ]:
def guesslatlonbounds(cube):
    cube.coord('projection_x_coordinate').circular = True

    cs = iris.coord_systems.GeogCS(6371229)
    for coord in ['projection_y_coordinate','projection_x_coordinate']:
        if cube.coord(coord).bounds is None:
            cube.coord(coord).guess_bounds()
        if cube.coord(coord).coord_system is None:
            cube.coord(coord).coord_system = cs
            
    return cube

In [ ]:
mask_1999_coords = guesslatlonbounds(newcube)

In [ ]:
#iris.save(subset_mask_1999_coords, '/data/users/hgilmour/initial_tracks/tobac_initial_tracks/segmentation/coords_bounds_test_segmentation_yearly_1999.nc')

**Process IMERG ready for re-gridding**

In [ ]:
### Load in IMERG file with 10km grid to compare to ###
imerg_file = '/scratch/hgilmour/obs/precip/precip_2001.nc'

imerg = iris.load(imerg_file)

imerg = imerg[0]

In [ ]:
## Take a small subset of imerg for testing ##

imerg_subset = imerg[0:1,:,:]

In [ ]:
imerg_subset

In [ ]:
def guesslatlonbounds(cube):
    cube.coord('longitude').circular = True

    cs = iris.coord_systems.GeogCS(6371229)
    for coord in ['latitude','longitude']:
        if cube.coord(coord).bounds is None:
            cube.coord(coord).guess_bounds()
        if cube.coord(coord).coord_system is None:
            cube.coord(coord).coord_system = cs
            
    return cube

In [ ]:
## check that imerg subset has bounds and a coordinate system ##

subset_imerg_coords = guesslatlonbounds(imerg_subset)

**Regrid the added coords and bounds cube to the IMERG 10km grid**

In [ ]:
scheme = iris.analysis.Nearest()

regridded_mask_1999 = mask_1999_coords.regrid(subset_imerg_coords, scheme)

In [ ]:
regridded_mask_1999

In [ ]:
## Save the regridded mask file as a test ##
savefile = '/data/users/hgilmour/initial_tracks/tobac_initial_tracks/segmentation/regridded/regridded_segmentation_yearly_1999.nc'

iris.save(regridded_mask_1999, savefile)

**Check regridded mask file by plotting it**

In [ ]:
import xarray as xr

In [ ]:
regrid_mask = xr.open_dataset('/data/users/hgilmour/initial_tracks/tobac_initial_tracks/segmentation/regridded/regridded_segmentation_yearly_1999.nc')

In [ ]:
regrid_mask = regrid_mask.segmentation_mask

In [ ]:
regrid_mask[8759,:,:].plot()